# LSTM for classification on Fashion MNIST

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
import matplotlib.pyplot as plt

In [2]:
#Let's download data using torchvision
trainset = datasets.FashionMNIST('./../0. Data/', 
                                 download = True, 
                                 train = True, 
                                 transform = T.Compose([
                                     T.ToTensor()
                                 ]))

testset = datasets.FashionMNIST('./../0. Data/', 
                                 download = True, 
                                 train = False, 
                                 transform = T.Compose([
                                     T.ToTensor()
                                 ]))

#split training data to training and validation  data
train_set, val_set = torch.utils.data.random_split(trainset, [50000, 10000])

#Convert data to dataloader
train_loader = torch.utils.data.DataLoader(train_set, 
                                          batch_size = 32, 
                                          shuffle = True)

val_loader = torch.utils.data.DataLoader(val_set,
                                        batch_size = 32,
                                        shuffle = True)

test_loader = torch.utils.data.DataLoader(testset, 
                                         batch_size = 32, 
                                         shuffle = True)

full_train_set  = torch.utils.data.DataLoader(trainset, 
                                          batch_size = 32, 
                                          shuffle = True)


In [3]:
#Check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
#Create  LSTM class
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        #save variables to use in other functions
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        #define LSTM layer
        self.lstm = nn.LSTM(input_size=input_size, 
                          hidden_size=hidden_size, 
                          num_layers=num_layers, 
                          batch_first=True)
        
        #convert output to desired output dimension(readout layer)
        self.fc = nn.Linear(in_features=hidden_size, out_features=output_size)
        
    def forward(self, x):
        #call LSTM layer
        x = x.view(x.shape[0],28,28)
        out, _ = self.lstm(x)
        
        #We will use only last output
        out = self.fc(out[:,-1,:])
        return out
        


In [5]:
#define training function
def train(Model, validate, max_epoch):
    for epoch in range(max_epoch):
        Train_Loss = []
        Val_Loss =[]
        loader = full_train_set
        
        if(validate):
            loader = train_loader
        
        cnf_tr = torch.zeros(10,10)
        cnf_val = torch.zeros(10,10)
        
        #Train on training data
        for i, sample in enumerate(loader):

            #set model to train mode
            Model.train()
            #set gradiuents to zero
            optimizer.zero_grad()
            #obtain output
            output = Model(sample[0].to(device)).to(device)
            #compute loss
            loss = loss_function(output, sample[1].to(device))
            #compute gradients
            loss.backward()
            #optimize weights
            optimizer.step()
            #record train loss
            Train_Loss.append(loss.item())
            
            with torch.no_grad():
                #calculate output by argmax
                output = torch.argmax(output, 1)
                #update entries in confusion matrix
                for i in range(output.shape[0]):
                    cnf_tr[output[i],sample[1][i]] +=1
            
        if(validate):
            #Evaluate on validation data
            with torch.no_grad():
                #set model to evaluation mode
                Model.eval()
                #evaluate on tvaidation data
                for i, sample in enumerate(val_loader):
                    output = Model(sample[0].to(device))
                    loss = loss_function(output, sample[1].to(device))
                    Val_Loss.append(loss.item())
                    #calculate output by argmax
                    output = torch.argmax(output, 1)
                    #update entries in confusion matrix
                    for i in range(output.shape[0]):
                        cnf_val[output[i],sample[1][i]] +=1
                   
        actual_count = torch.sum(cnf_tr, dim=0)
        correct_pred = torch.tensor([cnf_tr[i,i] for i in range(10)])
        A_tr = (torch.sum(correct_pred)/torch.sum(actual_count)).item()
        
        if(validate):
            actual_count = torch.sum(cnf_val, dim=0)
            correct_pred = torch.tensor([cnf_val[i,i] for i in range(10)])
            A_val = (torch.sum(correct_pred)/torch.sum(actual_count)).item()
        
        #print losses in every epoch
        if(validate):
            print('epoch : ',epoch,'; Train_acc : ', np.round(A_tr,4), '; Val_acc : ', np.round(A_val,4),  
                  '; Train_loss  ',np.round(np.mean(Train_Loss),4),  '; Val_loss  ',np.round(np.mean(Val_Loss),4))
        else:
            print('epoch = ',epoch,'; Train_acc : ', np.round(A_tr,4), '; Train_loss  ',np.round(np.mean(Train_Loss),4))

In [6]:
#Function top evaluate model using performace metrices
def evaluate(cnf):
    actual_count = torch.sum(cnf, dim=0)
    predicted_count = torch.sum(cnf, dim=1)
    correct_pred = torch.tensor([cnf[i,i] for i in range(10)])
    #Precision
    precision = correct_pred/predicted_count
    #Recall
    recall = correct_pred/actual_count
    #F1-Score
    f1_score = 2*precision*recall/(precision+recall)
    #Accuracy
    Accuracy = torch.sum(correct_pred)/torch.sum(actual_count)
    print('\n',pd.DataFrame({'Class':[i for i in range(10)],
                 'Precision' : precision,
                 'Recall' : recall,
                 'F1_Score': f1_score}))
    
    
    print('\nAccuracy  : ', Accuracy.item())

In [7]:
#function to test model
def test(Model):
    Loss = []
    #confusion matrix
    cnf = torch.zeros(10,10)

    #evaluate on test data
    with torch.no_grad():
        #set model to evaluation mode
        Model.eval()
        #evaluate on test data
        for i, sample in enumerate(test_loader):
            output = Model(sample[0].to(device))
            loss = loss_function(output, sample[1].to(device))
            Loss.append(loss.item())
            #calculate output by argmax
            output = torch.argmax(output, 1)
            #update entries in confusion matrix
            for i in range(output.shape[0]):
                cnf[output[i],sample[1][i]] +=1

        #print test loss
        print('Test loss : ', np.mean(Loss))

    #print evaluation summary
    evaluate(cnf)

In [8]:
#define loss function
loss_function = nn.CrossEntropyLoss()

In [9]:
#Create Model
Model = LSTM(input_size=28,
             hidden_size=64,
             num_layers=3,
             output_size=10).to(device)
#Define optimizer
optimizer = optim.Adam(Model.parameters())
#train model with validation
train(Model, validate=True, max_epoch=30)

epoch :  0 ; Train_acc :  0.7342 ; Val_acc :  0.8049 ; Train_loss   0.7277 ; Val_loss   0.5454
epoch :  1 ; Train_acc :  0.8192 ; Val_acc :  0.8351 ; Train_loss   0.49 ; Val_loss   0.461
epoch :  2 ; Train_acc :  0.8421 ; Val_acc :  0.8557 ; Train_loss   0.428 ; Val_loss   0.3977
epoch :  3 ; Train_acc :  0.855 ; Val_acc :  0.8688 ; Train_loss   0.387 ; Val_loss   0.3564
epoch :  4 ; Train_acc :  0.8642 ; Val_acc :  0.8647 ; Train_loss   0.3609 ; Val_loss   0.3665
epoch :  5 ; Train_acc :  0.8742 ; Val_acc :  0.8789 ; Train_loss   0.3367 ; Val_loss   0.338
epoch :  6 ; Train_acc :  0.8784 ; Val_acc :  0.883 ; Train_loss   0.3217 ; Val_loss   0.3179
epoch :  7 ; Train_acc :  0.8874 ; Val_acc :  0.8857 ; Train_loss   0.303 ; Val_loss   0.3127
epoch :  8 ; Train_acc :  0.8927 ; Val_acc :  0.8904 ; Train_loss   0.2889 ; Val_loss   0.3067
epoch :  9 ; Train_acc :  0.8957 ; Val_acc :  0.8873 ; Train_loss   0.2757 ; Val_loss   0.3098
epoch :  10 ; Train_acc :  0.9019 ; Val_acc :  0.8962 ; Tra

In [10]:
#Let's train our model for 21 epochs on full training set
#Create Model
Model = LSTM(input_size=28,
                    hidden_size=64,
                    num_layers=3,
                    output_size=10).to(device)

#Define optimizer
optimizer = optim.Adam(Model.parameters())
#train
train(Model, validate=False, max_epoch=21)

epoch =  0 ; Train_acc :  0.7417 ; Train_loss   0.7122
epoch =  1 ; Train_acc :  0.8301 ; Train_loss   0.4643
epoch =  2 ; Train_acc :  0.8546 ; Train_loss   0.3926
epoch =  3 ; Train_acc :  0.8692 ; Train_loss   0.3569
epoch =  4 ; Train_acc :  0.8761 ; Train_loss   0.3346
epoch =  5 ; Train_acc :  0.8834 ; Train_loss   0.3145
epoch =  6 ; Train_acc :  0.8894 ; Train_loss   0.2981
epoch =  7 ; Train_acc :  0.8941 ; Train_loss   0.2856
epoch =  8 ; Train_acc :  0.9011 ; Train_loss   0.27
epoch =  9 ; Train_acc :  0.9024 ; Train_loss   0.2592
epoch =  10 ; Train_acc :  0.907 ; Train_loss   0.2486
epoch =  11 ; Train_acc :  0.91 ; Train_loss   0.2394
epoch =  12 ; Train_acc :  0.9139 ; Train_loss   0.231
epoch =  13 ; Train_acc :  0.9171 ; Train_loss   0.2232
epoch =  14 ; Train_acc :  0.9199 ; Train_loss   0.2153
epoch =  15 ; Train_acc :  0.9233 ; Train_loss   0.2066
epoch =  16 ; Train_acc :  0.9254 ; Train_loss   0.1984
epoch =  17 ; Train_acc :  0.9283 ; Train_loss   0.1919
epoch = 

In [11]:
#Let's test model now
test(Model)

Test loss :  0.2894720965728592

    Class  Precision  Recall  F1_Score
0      0   0.822688   0.863  0.842362
1      1   0.977137   0.983  0.980060
2      2   0.841286   0.811  0.825866
3      3   0.900802   0.899  0.899900
4      4   0.818006   0.845  0.831284
5      5   0.980962   0.979  0.979980
6      6   0.743697   0.708  0.725410
7      7   0.946654   0.976  0.961103
8      8   0.986882   0.978  0.982421
9      9   0.975460   0.954  0.964611

Accuracy  :  0.8996000289916992


In [12]:
#Finally, let's save our model
torch.save(Model.state_dict(), './saved_models/LSTM_FMNIST.pth')

In [ ]:
#To Retrieve
Modelx = LSTM(input_size=28,
                    hidden_size=64,
                    num_layers=3,
                    output_size=10).to(device)
Modelx.load_state_dict(torch.load('./saved_models/LSTM_FMNIST.pth'))